In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from math import nan
import glob
import dask
import datetime
from dateutil.relativedelta import relativedelta
from functools import partial
import pandas as pd
import sys
from smyleutils import readdata_utils as read
import isla_interp_utils as isla_interp
from smyleutils import filter_utils as filt

dask.config.set(**{'array.slicing.split_large_chunks': True})

### Set up info

In [2]:
ystart = 1970 # start year of hindcasts 
yend = 2020 # end year of hindcasts
nyears = yend - ystart + 1
initmon=2
initmonstr = str(initmon).zfill(2)
nmems = 20 # the number of hindcast members
memstr = [str(i).zfill(3) for i in np.arange(1,nmems+1,1)] # generating member strings
topdir="/glade/campaign/cesm/development/espwg/SMYLE-CW3E/timeseries/"
#expname = "b.e21.BSMYLE-CW3E-L83"
expname="b.e21.BSMYLE-CW3E.f09_g17."
outpath="/glade/campaign/cgd/cas/islas/python_savs/NCAR_CW3E_SMYLE/DATA_SORT/U_TEJ/"

### Set up dask cluster

In [3]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '30GB',
    processes = 1,
    queue = 'casper',
    local_directory='$TMPDIR',
    resource_spec='select=1:ncpus=1:mem=30GB',
    project='P04010022',
    walltime='02:00:00',
    interface='ext')

# scale up
cluster.scale(24)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

/glade/u/home/islas/miniconda3/envs/islaenv24/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44077 instead
  f"Port {expected} is already in use.\n"


In [4]:
cluster

PBSCluster(8dca1f2b, 'tcp://128.117.208.210:46011', workers=24, threads=24, memory=670.56 GiB)

In [5]:
client

<Client: 'tcp://128.117.208.210:46011' processes=24 threads=24, memory=670.56 GiB>

### Set up preprocessor

In [6]:
def fixtime(dat):
    diff = np.array(dat.time_bnds.isel(nbnd=1)) - np.array(dat.time_bnds.isel(nbnd=0))
    diff = diff/2.
    newtime = np.array(dat.time_bnds.isel(nbnd=0)) + diff
    dat['time'] = newtime
    return dat

In [7]:
### pre-processor to ensure all initialization dates have the same time axis
def preprocessor(ds):
    time = fixtime(ds)
    datestart = pd.to_datetime("1970-"+str(ds.isel(time=0).time.dt.month.values).zfill(2)+"-"+str(ds.isel(time=0).time.dt.day.values).zfill(2), format="%Y-%m-%d")
    time = [ datestart + relativedelta(months=int(i)) for i in np.arange(0,ds.time.size,1) ]
    ds['time'] = time
    return ds

### Read in the geopotential height

In [8]:
phis = xr.open_dataset("/glade/campaign/cesm/collections/CESM2-LE/atm/proc/tseries/month_1/PHIS/"
     +"b.e21.BHISTcmip6.f09_g17.LE2-1001.001.cam.h0.PHIS.185001-185912.nc").isel(time=0).load()

### Loop over years, interpolate in the vertical

In [11]:
ufiles

['/glade/campaign/cesm/development/espwg/SMYLE-CW3E/timeseries/b.e21.BSMYLE-CW3E.f09_g17.1971-02.001/atm/proc/tseries/month_1/b.e21.BSMYLE-CW3E.f09_g17.1971-02.001.cam.h0.U.197102-197201.nc',
 '/glade/campaign/cesm/development/espwg/SMYLE-CW3E/timeseries/b.e21.BSMYLE-CW3E.f09_g17.1971-02.002/atm/proc/tseries/month_1/b.e21.BSMYLE-CW3E.f09_g17.1971-02.002.cam.h0.U.197102-197201.nc',
 '/glade/campaign/cesm/development/espwg/SMYLE-CW3E/timeseries/b.e21.BSMYLE-CW3E.f09_g17.1971-02.003/atm/proc/tseries/month_1/b.e21.BSMYLE-CW3E.f09_g17.1971-02.003.cam.h0.U.197102-197201.nc',
 '/glade/campaign/cesm/development/espwg/SMYLE-CW3E/timeseries/b.e21.BSMYLE-CW3E.f09_g17.1971-02.004/atm/proc/tseries/month_1/b.e21.BSMYLE-CW3E.f09_g17.1971-02.004.cam.h0.U.197102-197201.nc',
 '/glade/campaign/cesm/development/espwg/SMYLE-CW3E/timeseries/b.e21.BSMYLE-CW3E.f09_g17.1971-02.005/atm/proc/tseries/month_1/b.e21.BSMYLE-CW3E.f09_g17.1971-02.005.cam.h0.U.197102-197201.nc',
 '/glade/campaign/cesm/development/espwg

In [12]:
allyears=[]
for iyear in np.arange(ystart,yend+1,1):
    print(iyear)
    ufiles = sorted([  glob.glob(topdir+expname+str(iyear)+'-'+str(initmon).zfill(2)+'.'+str(imem).zfill(3)+"/atm/proc/tseries/month_1/*.cam.h0.U.*.nc")[0] 
                for imem in np.arange(1,nmems+1,1) ])
    psfiles = sorted([  glob.glob(topdir+expname+str(iyear)+'-'+str(initmon).zfill(2)+'.'+str(imem).zfill(3)+"/atm/proc/tseries/month_1/*.cam.h0.PS.*.nc")[0] 
                for imem in np.arange(1,nmems+1,1) ])

    #---A couple of members are corrupted for Feb initialization so I'm replacing them with adjeacent members for now
    if initmon == 2:
        if (iyear == 1971):
            ufiles[6] = ufiles[5]
            psfiles[6] = psfiles[5]
        if (iyear == 1975):
            ufiles[3] = ufiles[2]
            psfiles[3] = psfiles[2]
    
    allulev=[]
    for imem in np.arange(0,len(ufiles),1):
        u = xr.open_mfdataset(ufiles[imem],
                          parallel=True, data_vars=['U'], coords='minimal', compat='override', preprocess = partial(preprocessor))
        ps = xr.open_mfdataset(psfiles[imem],
                           parallel=True, data_vars=['PS'], coords='minimal', compat='override', preprocess = partial(preprocessor))

        uuse = u.U.sel(lon=120., method='nearest')
        phiuse = phis.sel(lon=120., method='nearest')
        psuse = ps.PS.sel(lon=120., method='nearest')
        newlevs=np.array([400., 350., 300., 250., 225., 175., 150., 125., 100.])*100.

        u_plev = isla_interp.interp_hybrid_to_pressure(
            uuse, psuse, u.hyam, u.hybm, p0=1e5, new_levels=newlevs, method='log',
            lev_dim='lev', extrapolate=False, variable='other',t_bot = uuse.isel(lev=u.lev.size-1), phi_sfc = phis)
        u_plev = u_plev.load()
        allulev.append(u_plev)
    allulev = xr.concat(allulev, dim='M')
    allyears.append(allulev)
allyears = xr.concat(allyears, dim='init_year')
allyears.to_netcdf(outpath+'U_400_100_120E_L32_init'+initmonstr+'.nc')

1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [13]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
ERROR:asyncio:_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


In [16]:
imem=1
print(topdir+expname+str(iyear)+'-'+str(initmon)+'.'+str(imem).zfill(3)+"/atm/proc/tseries/month_1/*.cam.h0.U.*.nc")

/glade/campaign/cesm/development/espwg/SMYLE-CW3E-L83/timeseries/b.e21.BSMYLE-CW3E-L83.f09_g17.1970-2.001/atm/proc/tseries/month_1/*.cam.h0.U.*.nc
